This notebook performs the optimization task to find the location where a new grocery store should be placed, to maximize access

How this works (implementation):

1. Use the helper_population_allocation.py to allocate a population count to each residential building (not ready yet, some placeholder function in there)
2. Use the helper_distance_calculation.py to calculate existing access and distance between a residential and commercial building
3. Once 1 and 2 are done, all parameters are ready. 

4. Then this notebook does some pre optimization setup and then runs an optimization model in Gurobi. It's pretty anti climatic. 

Next steps:

- Update the population allocation helper file
- Figure out the correct ordering of arrays. It's not accurate currently 


In [1]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
import gurobipy as gp
from gurobipy import GRB

# Helper modules
import helper_population_allocation as pa
import helper_distance_calculation as dc

# Avoid printing set copy warnings
import warnings
warnings.filterwarnings("ignore")



PRE-OPTIMIZATION SETUP

In [2]:
%%time

# Get the main buildings dataset 
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')

# Create ID variable
buildings_df.reset_index(drop=True, inplace=True)
buildings_df['building_id'] = buildings_df.index + 1
buildings_df['building_id'] = buildings_df.apply(lambda row: str(row['building_id']) + '-' + str(row['CLASS']) , axis=1)

# buildings_df = buildings_df.sample(n=2000, random_state=1)  # Remove later




Wall time: 8.75 s


In [3]:
# Create arrays to track ordering (residential)
res_buildings = buildings_df[buildings_df['class_reco'].str.contains('Residential')]
res_buildings = res_buildings.sort_values('building_id')
res_buildings = dc.get_geocoordinate(res_buildings, 'geometry')

res_buildings_array = np.array(res_buildings['building_id'])
res_buildings_coordinates_array = np.array(res_buildings['coordinates'])

In [4]:
# Create arrays to track ordering (Commercial)
comm_buildings = buildings_df[buildings_df['class_reco'].str.contains('commercial')]
comm_buildings = comm_buildings.sort_values('building_id')
comm_buildings = dc.get_geocoordinate(comm_buildings, 'geometry')

comm_buildings_array = np.array(comm_buildings['building_id'])
comm_buildings_coordinates_array = np.array(comm_buildings['coordinates'])


In [5]:
# Create arrays to track ordering (grocery stores)
grocery_stores = buildings_df[buildings_df['class_reco'].str.contains('Grocery')]
grocery_stores = grocery_stores.sort_values('building_id')
grocery_stores = dc.get_geocoordinate(grocery_stores, 'geometry')

grocery_stores_array = np.array(grocery_stores['building_id'])
grocery_stores_coordinates_array = np.array(grocery_stores['coordinates'])


In [86]:
%%time
# DONT RUN THIS AGAIN

# Create parameter matrices (Res comm access matrix - Bij)
# [i,j] value indicates whether residential building i is within access distance of commercial building j
# res_comm_distance_matrix, res_comm_access_matrix = dc.calculate_access(res_buildings_coordinates_array, comm_buildings_coordinates_array)

# # Save file
# np.save('res_comm_access_matrix', res_comm_access_matrix)
# np.save('res_comm_distance_matrix', res_comm_distance_matrix)


Wall time: 0 ns


In [6]:
# Load the files and use it 
res_comm_access_matrix = np.load('res_comm_access_matrix.npy')
res_comm_distance_matrix = np.load('res_comm_distance_matrix.npy')

In [7]:
%%time

# Create parameter matrices (Res groc access array - Aj)
# ith value indicates whether the ith residential building has existing access
res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)
res_access_array = np.amax(res_groc_access_matrix, 1)


Wall time: 25.8 s


In [8]:
%%time

# Create parameter matrices (Res Population - Pj)
# ith value indicates the population in the ith column
res_population = pa.get_population(geopandas_dataframe=res_buildings) 
res_population_array = np.array(res_population['population'])
res_population_array

Wall time: 1.59 s


array([1, 1, 1, ..., 1, 2, 2], dtype=int64)

OPTIMIZATION

Steps:
i = set of all commercial buildings
j = set of all residential buildings

Decision variable:
- Ci = 1 if the new grocery store is put in commercial building i, 0 otherwise

Parameters:
- Aj = 1 if residential building j already has access to a food store (within 1 mile)
- Bij = 1 if commercial building i is within 1 mile of residential building j
- Pj = Population at building j

Objective function:

Max $$ \sum_i \sum_j (1-A_j)*P_j*B_{ij}*C_i$$

Constraint (only 1 grocery store location being allocated):
$$ \sum_i C_i = 1$$





In [11]:
# Gurobi takes forever to optimize since it relies on for loops. So we are going to do the optimization manually. 

###########################
# STEP 1: Take the res_comm_access_matrix, remove those rows (each row represents a residential building) which have existing access
###########################
existing_access_indices = res_access_array.nonzero()[0] # These are indices of residential buildings that currently have access
res_comm_access_matrix_subset = np.delete(res_comm_access_matrix, existing_access_indices, axis=0 )

###########################
# STEP 2: Do the same thing for res_population_array so that the ordering matches
###########################
res_population_array_sub = np.delete(res_population_array, existing_access_indices, axis=0)

###########################
# STEP 3: Do a matrix multiplication between res_population_array_sub and res_comm_access_matrix_sub
###########################

# How this works:

# 1. Reshape res_population_array_sub to be (1 * 2780) 2D array
# 2. res_comm_access_matrix_sub is (2780 * 6895)
# 3. When you do matrix multiplication of 1 and 2, you get a (1*6895) array
# 4. Each element of this array would represent the sum of the population at each residential building multiplied by whether that residential building and that particular commercial building
# are within access region. So for example, first element of this result would be P0 * whether res building 0 and comm building 0 are within access + P1 * whether res building 1 and comm building 0 are within access and so on
# So each element of the result represents the total new population that would gain access if a commercial building is put at that index

res_population_array_sub = np.reshape(res_population_array_sub, (-1, len(res_population_array_sub)))
new_access_array = np.matmul(res_population_array_sub, res_comm_access_matrix_subset)

###########################
# STEP 4: Print results
###########################
chosen_comm_index = np.argmax(new_access_array)
chosen_comm_building = comm_buildings_array[chosen_comm_index]
new_access_created = np.max(new_access_array)

print(f"the new store should be put at commercial building {chosen_comm_building}")
print(f"putting the store here would give access to {new_access_created} new people")


the new store should be put at commercial building 2816-R
putting the store here would give access to 2723 new people


In [14]:
# Now write the optimization that can put n stores in a greedy way

# Process - do the exact same thing as above, but in a loop. Keep updating the access array as you place each commercial building

def place_n_stores(num_stores, access_matrix, access_array):

    store_indices = []
    store_ids = []
    marginal_access_gain = []

    res_access_array_copy = access_array.copy()
    for n in range(num_stores):

        if np.sum(res_access_array_copy) == len(res_buildings_array): # This means every building now has access
            pass
        else:
            # STEP 1
            existing_access_indices = res_access_array_copy.nonzero()[0] # These are indices of residential buildings that currently have access
            res_comm_access_matrix_subset = np.delete(access_matrix, existing_access_indices, axis=0 )

            # STEP 2
            res_population_array_sub = np.delete(res_population_array, existing_access_indices, axis=0)

            # STEP 3
            res_population_array_sub = np.reshape(res_population_array_sub, (-1, len(res_population_array_sub)))
            new_access_array = np.matmul(res_population_array_sub, res_comm_access_matrix_subset)

            chosen_comm_index = np.argmax(new_access_array)
            chosen_comm_building = comm_buildings_array[chosen_comm_index]
            new_access_created = np.max(new_access_array)

            # STEP 4: Update results and arrays
            
            # Which residential buildings does this new store give access to
            new_buildings_with_access = access_matrix[:,chosen_comm_index].nonzero()[0]    # These are the indices in the res_access_array that need to be replaced (these buildings now have access)

            # Update the access values of these buildings in the access array
            replace_vals = list(np.ones(new_buildings_with_access.shape)) # These are the values which with certain elements of the access array will be replaced with

            # Perform replace
            res_access_array_copy[new_buildings_with_access] = replace_vals

            # STEP 5: Store results
            store_indices.append(chosen_comm_index)
            store_ids.append(chosen_comm_building)
            marginal_access_gain.append(new_access_created)


    return store_indices, store_ids, marginal_access_gain





In [21]:
store_indices, store_ids, marginal_access_gain = place_n_stores(5, access_array=res_access_array, access_matrix=res_comm_access_matrix)

print(f"Num additional stores needed to fill complete access = {len(store_indices)}")

results_df = pd.DataFrame(list(zip(store_ids, marginal_access_gain)),
                                columns = ['building_id', 'marginal_access_gain'])
                                
results_df


Num additional stores needed to fill complete access = 2


,building_id,marginal_access_gain
0,2816-R,2723
1,116276-C,880


In [22]:
# WHAT IF THERE WERE NO GROCERY STORES TO BEGIN WITH. HOW MANY MINIMUM NUMBER OF STORES ARE NEEDED AND WHERE TO COMPLETE ACCESS

res_access_array_zero = np.zeros(res_access_array.shape)

store_indices, store_ids, marginal_access_gain = place_n_stores(50, access_array=res_access_array_zero, access_matrix=res_comm_access_matrix)

print(f"Num additional stores needed to fill complete access = {len(store_indices)}")

results_df = pd.DataFrame(list(zip(store_ids, marginal_access_gain)),
                                columns = ['building_id', 'marginal_access_gain'])
                                
results_df

Num additional stores needed to fill complete access = 15


,building_id,marginal_access_gain
0,94210-C,52684
1,25462-C,31990
2,78124-C,28162
3,70644-C,27096
4,116123-C,23021
5,81793-C,14343
6,3960-C,4720
7,52096-C,4196
8,22083-C,3483
9,96804-C,2138


In [25]:
# CHANGING THE ACCESS DEFINITION TO 0.5 MILES FROM 1 MILE

# Import res comm access matrix again (just to be safe)
res_comm_access_matrix = np.load('res_comm_access_matrix.npy')
res_comm_distance_matrix = np.load('res_comm_distance_matrix.npy')

# Creating a modified res comm access matrix 
res_comm_access_matrix_half_mile = res_comm_distance_matrix.copy()
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile <= 0.5] = 1
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile != 1] = 0

# Create a modified res existing access array
res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)

res_groc_access_matrix_half_mile = res_groc_distance_matrix.copy()
res_groc_access_matrix_half_mile[res_groc_access_matrix_half_mile <= 0.5] = 1
res_groc_access_matrix_half_mile[res_groc_access_matrix_half_mile != 1] = 0

res_access_array_half_mile = np.amax(res_groc_access_matrix_half_mile, 1)


# Calculate 
store_indices, store_ids, marginal_access_gain = place_n_stores(50, access_array=res_access_array_half_mile,
                                                                     access_matrix=res_comm_access_matrix_half_mile)

print(f"Num additional stores needed to fill complete access = {len(store_indices)}")

results_df = pd.DataFrame(list(zip(store_ids, marginal_access_gain)),
                                columns = ['building_id', 'marginal_access_gain'])
                                
results_df




Num additional stores needed to fill complete access = 21


,building_id,marginal_access_gain
0,19415-C,5432.0
1,31851-C,4302.0
2,105592-R,4206.0
3,3018-R,2851.0
4,110387-C,2531.0
5,14692-C,2032.0
6,65810-C,1880.0
7,109-C,937.0
8,3898-C,915.0
9,2877-C,596.0


In [26]:
# IF THERE WERE NO EXISTING STORES
# Import res comm access matrix again (just to be safe)
res_comm_access_matrix = np.load('res_comm_access_matrix.npy')
res_comm_distance_matrix = np.load('res_comm_distance_matrix.npy')

# Creating a modified res comm access matrix 
res_comm_access_matrix_half_mile = res_comm_distance_matrix.copy()
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile <= 0.5] = 1
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile != 1] = 0

res_access_array_zero = np.zeros(res_access_array.shape)


store_indices, store_ids, marginal_access_gain = place_n_stores(100, access_array=res_access_array_zero,
                                                                     access_matrix=res_comm_access_matrix_half_mile)

print(f"Num additional stores needed to fill complete access = {len(store_indices)}")

results_df = pd.DataFrame(list(zip(store_ids, marginal_access_gain)),
                                columns = ['building_id', 'marginal_access_gain'])
                                
results_df






Num additional stores needed to fill complete access = 49


,building_id,marginal_access_gain
0,112491-C,30762.0
1,103358-C,20846.0
2,101372-C,13882.0
3,61696-C,13213.0
4,92814-C,12869.0
5,114016-C,12010.0
6,86911-C,10271.0
7,80815-C,9926.0
8,19741-C,8737.0
9,52332-C,8449.0


In [ ]:
# Plot 1 mile and half mile
